# CKO Revision

Compustat firms over 2000 and 2017 that meet the following criteria
- All controls including Ravenpack
- Delete 6000 <= SIC <= 6999 and utilities
- Should be in the HP dataset

Merge in with SDC
- Should be in Linking table
- Merge with SDC: (a) 50% > shares purchased in transaction and (b) Complete
- Should have (a) Target LT sales (b) Target market value 4 weeks prior to announcements (c) Deal values (d) Target total assets

### Import modules

In [1]:
import pandas as pd
import numpy as np
import rpy2.rinterface #ggplot tool
from pandas_profiling import ProfileReport
import dask.dataframe as dd
import wrds
import pandasql as ps
import sqlite3

In [ ]:
# ## Review TNIC-3 data

# ### Import TNIC3 data from Hoberg and Philips data library 

# # !wget -P ../2_pipeline/ http://hobergphillips.tuck.dartmouth.edu/idata/tnic3_data.zip
# # !unzip -q ../2_pipeline/tnic3_data.zip -d ../2_pipeline/ && rm ../2_pipeline/tnic3_data.zip

# """
# Hoberg and Philips TNIC3 database
# """
# tnic = pd.read_csv('/Users/ohn0000/Dropbox/Project/cko/0_data/external/tnic3_data.txt', 
#                    delimiter='\t', header=0, index_col=['gvkey1', 'year', 'gvkey2'])
# tnic.dropna(inplace=True)

# ### Subset to 20-closest competitors

# # tnic_industry = tnic.groupby(level=['gvkey1', 'year'])["score"].nlargest(20).reset_index(level=[0,1], drop=True)
# # tnic_industry = tnic_industry.to_frame(name='score')
# # tnic_industry.to_pickle('../2_pipeline/tnic_industry.pkl')
# tnic_industry = pd.read_pickle('../2_pipeline/tnic_industry.pkl')

# ```tnic_industry``` still has firm-years with less than 20 competitors.

# # """
# # Require at least 20 closest competitors
# # """
# # tnicind_sub = tnic.groupby(level=['gvkey1', 'year'])["score"].filter(lambda x: x.size == 20)
# # tnicind_sub = tnicind_sub.to_frame(name='score')

# """
# tnic_industry['gvkey1'] = tnic_industry['gvkey1'].apply(lambda x: str(x).zfill(6))
# tnic_industry['gvkey2'] = tnic_industry['gvkey2'].apply(lambda x: str(x).zfill(6))
# """

# Remeber that _year_ in __tnic_industry__ is the base year for identifying close competitors. Accordingly, _lead1_ is the M&A year and _lead2_ is the year following M&A.

# Readme_tnic3.txt explains that _year_ equals the first four digits of the __compustat__ _datadate_.

# ### Shift years in __tnic_industry__ to get _lead1_ and _lead2_ similarity scores

# tnic_industry.rename(columns={'score':'score_0'}, inplace=True)

# for i in range(1,3):
#     colname = 'score' + '_' + str(i)
#     tnic_industry['score'] = np.NaN
#     tnic_industry.index = tnic_industry.index.set_levels(tnic_industry.index.levels[1] + 1, level=1)
#     tnic_industry.update(tnic)
#     tnic_industry.rename(columns={'score':colname}, inplace=True)

# tnic_industry.reset_index(inplace=True)
# tnic_industry["year"] -= 2
# tnic_industry.set_index(["gvkey1", "year", "gvkey2"], inplace=True)

# tnic_industry.to_pickle('../2_pipeline/tnic_industry.pkl')

# ### Run __*tnic_industry.py*__ on _WRDS_ to update lead scores

# Many of the _lead1_ and _lead2_ values are missing. Grab these values from __TNIC_Advanced__ uploaded on _WRDS_. 

# # !scp ../2_pipeline/tnic_industry.pkl tnic_industry.py $WRDS:~

# ### Download updated __*tnic_industry*__ file from WRDS

# # !scp $WRDS:/scratch/ou/hohn/tnic_ind_update.pkl ../2_pipeline/ 

In [2]:
tnic_industry = pd.read_pickle('../2_pipeline/tnic_ind_update.pkl')

In [3]:
tnic_10 = tnic_industry.groupby(level=['gvkey1', 'year'])["score_0"].nlargest(10).reset_index(level=[0,1], drop=True).to_frame()

Redefine _tnic_industry_ as below to subset to 10-closest competitors.

In [4]:
tnic_industry = tnic_10.join(tnic_industry[['score_1','score_2']])

Average TNIC similarity score across closest competitors.  
Remeber that in __TNIC_ALL__ most of the scores equals to zero. The _z\__ might be the more suitable.
- Close pair in t0 not appearing in t1 or t2 is meaningful.
- __BE CAREFUL__ of year 2016 and 2017. __TNIC is available only up to 2017__.

In [5]:
avg_sim = tnic_industry.groupby(level=['gvkey1','year']).mean()
avg_sim = avg_sim.join(tnic_industry.groupby(level=['gvkey1','year']).count().add_prefix("n_"))
avg_sim = avg_sim.join(tnic_industry.fillna(0).groupby(level=['gvkey1','year']).mean().add_prefix("z_"))

Calculate _avg_sim_ of competitors

In [6]:
ind_avg = tnic_industry.join(avg_sim.reset_index().rename(columns={'gvkey1':'gvkey2'}).set_index(['gvkey2', 'year']), 
                   on=['gvkey2', 'year'], how='left', rsuffix='_a').groupby(level=['gvkey1','year']).mean()
ind_avg = ind_avg[['score_0_a', 'score_1_a', 'score_2_a', 'z_score_0', 'z_score_1', 'z_score_2']]
ind_avg.rename(columns={'score_0_a':'s_0_a', 
                        'score_1_a':'s_1_a',
                        'score_2_a':'s_2_a',
                        'z_score_0':'z_0_a', 
                        'z_score_1':'z_1_a',
                        'z_score_2':'z_2_a'}, inplace=True)

In [7]:
avg_sim = avg_sim.join(ind_avg)

Total observation in _avg_sim_ and those with 20 competitors in t0

In [8]:
print(len(avg_sim), len(avg_sim[avg_sim['n_score_0']==10]))

109791 84794


In [9]:
avg_sim.head()

score_0   score_1   score_2  n_score_0  n_score_1  n_score_2  \
gvkey1 year                                                                 
1004   1995  0.00580  0.123100  0.069600          1          1          1   
       1996  0.02083  0.043288  0.048657         10          8          7   
       1997  0.04576  0.027556  0.037771         10          9          7   
       1998  0.02231  0.026100  0.042560         10          6          5   
       1999  0.03026  0.032500  0.047910         10         10         10   

             z_score_0  z_score_1  z_score_2     s_0_a     s_1_a     s_2_a  \
gvkey1 year                                                                  
1004   1995    0.00580    0.12310    0.06960       NaN       NaN       NaN   
       1996    0.02083    0.03463    0.03406  0.046411  0.050805  0.047070   
       1997    0.04576    0.02480    0.02644  0.050885  0.062084  0.057949   
       1998    0.02231    0.01566    0.02128  0.034312  0.043136  0.050472   
       1999    0.03026    0.03250    0.04791  0.041202  0.053419  0.051189   

                z_0_a     z_1_a     z_2_a  
gvkey1 year                                
1004   1995       NaN       NaN       NaN  
       1996  0.046411  0.036585  0.027966  
       1997  0.050885  0.056042  0.035902  
       1998  0.034312  0.020987  0.012926  
       1999  0.041202  0.041825  0.036488

### Grab COMPUSTAT _datadate_

Revise here if _avg\_sim_ needs additional COMPUSTAT variables

In [10]:
db = wrds.Connection()

Enter your WRDS username [ohn0000]: hohn
Enter your password: ········


WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [11]:
comp_query = """
select distinct gvkey, datadate, fyear, indfmt, sale
from comp.funda
where consol = %(consol)s and indfmt in %(indfmt)s 
    and datafmt = %(datafmt)s and popsrc = %(popsrc)s
    and curcd in %(curcd)s
order by gvkey, fyear
"""

parm = {'consol':('C'), 'indfmt' : ('INDL', 'FS'), 'datafmt': ('STD'), 'popsrc' : ('D'), 'curcd' : ('USD', 'CAD')}
comp = db.raw_sql(comp_query, date_cols=['datadate'], params=parm)

In [12]:
comp['fyear'] = comp['fyear'].astype('int16')
comp['year'] = comp['datadate'].dt.year.astype('int16')

comp['gvkey1'] = pd.to_numeric(comp['gvkey']).astype('int64')
comp.drop(columns='gvkey', inplace=True)

comp.set_index(['gvkey1', 'fyear'], inplace=True)

Not sure how TNIC deals fiscal years where fyear are differenct but
year are the same. Assume the lastest datadate within _year_

In [13]:
datadate = comp[~comp.index.duplicated(keep='last')][['datadate', 'year']].reset_index()

In [14]:
avg_sim = pd.merge(avg_sim, datadate, 
                 left_index=True, 
                 right_on=['gvkey1', 'year'], how='left').drop_duplicates(['gvkey1', 'year'], keep='last')

In [15]:
avg_sim = avg_sim[avg_sim['datadate'].notnull()]

TNIC data not missing _datadate_

In [16]:
len(avg_sim[lambda x : (x['fyear']>=2000)&(x['fyear']<=2017)&(x['n_score_0']==10)])

62596

In [17]:
col = list(avg_sim)
col = col[-4:] + col[:-4]
avg_sim = avg_sim[col].sort_values(['gvkey1', 'year', 'datadate'])

In [18]:
avg_sim.head()

,gvkey1,fyear,datadate,year,score_0,score_1,score_2,n_score_0,n_score_1,n_score_2,z_score_0,z_score_1,z_score_2,s_0_a,s_1_a,s_2_a,z_0_a,z_1_a,z_2_a
77,1004,1994.0,1995-05-31,1995,0.00580,0.123100,0.069600,1,1,1,0.00580,0.12310,0.06960,NaN,NaN,NaN,NaN,NaN,NaN
78,1004,1995.0,1996-05-31,1996,0.02083,0.043288,0.048657,10,8,7,0.02083,0.03463,0.03406,0.046411,0.050805,0.047070,0.046411,0.036585,0.027966
79,1004,1996.0,1997-05-31,1997,0.04576,0.027556,0.037771,10,9,7,0.04576,0.02480,0.02644,0.050885,0.062084,0.057949,0.050885,0.056042,0.035902
80,1004,1997.0,1998-05-31,1998,0.02231,0.026100,0.042560,10,6,5,0.02231,0.01566,0.02128,0.034312,0.043136,0.050472,0.034312,0.020987,0.012926
81,1004,1998.0,1999-05-31,1999,0.03026,0.032500,0.047910,10,10,10,0.03026,0.03250,0.04791,0.041202,0.053419,0.051189,0.041202,0.041825,0.036488


### Grab _permno_

Revise here if _avg\_sim_ needs additional CRSP variables

In [19]:
sql_query= """
select gvkey, liid as iid, lpermno as permno, linkdt, linkenddt
from crsp.ccmxpf_linktable
where linktype in %(type)s and linkprim in %(prim)s and usedflag = 1
"""

parm = {'type':('LU', 'LC'), 'prim':('P', 'C')}
linktable = db.raw_sql(sql_query, date_cols=['linkdt', 'linkenddt'], params=parm)

linktable['gvkey'] = pd.to_numeric(linktable['gvkey'])
linktable['permno'] = pd.to_numeric(linktable['permno']).astype('int64')
linktable['iid'] = linktable['iid'].astype('category')

In [20]:
enddt = pd.to_datetime('2020-01-07 00:00:00')

In [21]:
linktable['linkenddt'] = linktable['linkenddt'].fillna(value = enddt)
linktable['linkenddt'] = linktable['linkenddt'].dt.date

In [22]:
conn = sqlite3.connect(':memory:')
avg_sim.to_sql('avg_sim', conn, index=False)
linktable.to_sql('linktable', conn, index=False)

qry = '''
    select  
        avg_sim.*, linktable.permno
    from
        avg_sim left join linktable on
        avg_sim.datadate between linkdt and linkenddt and avg_sim.gvkey1 = linktable.gvkey
    '''
df = pd.read_sql_query(qry, conn)
df['permno'] = df['permno'].astype('Int64')
df['datadate'] = df['datadate'].astype('datetime64[ns]')

df[df['permno'].isna()].to_sql('df', conn, index=False)

qry = '''
    select
        a.*, b.gvkey1, b.datadate
        from linktable a join df b
        on 
            a.gvkey = b.gvkey1
'''
aug = pd.read_sql_query(qry, conn)

aug.to_sql('aug', conn, index=False)
qry = '''
    select gvkey1, permno, iid, min(linkdt) as linkdt, max(linkenddt) as linkenddt
    from aug
    group by gvkey1, permno, iid
    order by gvkey1, linkdt
'''

df = df.merge(aug[aug['iid'].isin(['01','02'])].rename(columns={'permno':'permno1'})[['gvkey1', 'permno1']], 
         left_on = ['gvkey1'], right_on=['gvkey1'], how='left')
df['permno'] = np.where(df['permno'].isna(), df['permno1'], df['permno'])

In [23]:
df = df.drop_duplicates(['gvkey1', 'year']).drop(columns='permno1')

In [24]:
col = list(df)
col.insert(2, col.pop(col.index('permno')))
df = df[col]

In [25]:
df.head()

,gvkey1,fyear,permno,datadate,year,score_0,score_1,score_2,n_score_0,n_score_1,n_score_2,z_score_0,z_score_1,z_score_2,s_0_a,s_1_a,s_2_a,z_0_a,z_1_a,z_2_a
0,1004,1994.0,54594,1995-05-31,1995,0.00580,0.123100,0.069600,1,1,1,0.00580,0.12310,0.06960,NaN,NaN,NaN,NaN,NaN,NaN
1,1004,1995.0,54594,1996-05-31,1996,0.02083,0.043288,0.048657,10,8,7,0.02083,0.03463,0.03406,0.046411,0.050805,0.047070,0.046411,0.036585,0.027966
2,1004,1996.0,54594,1997-05-31,1997,0.04576,0.027556,0.037771,10,9,7,0.04576,0.02480,0.02644,0.050885,0.062084,0.057949,0.050885,0.056042,0.035902
3,1004,1997.0,54594,1998-05-31,1998,0.02231,0.026100,0.042560,10,6,5,0.02231,0.01566,0.02128,0.034312,0.043136,0.050472,0.034312,0.020987,0.012926
4,1004,1998.0,54594,1999-05-31,1999,0.03026,0.032500,0.047910,10,10,10,0.03026,0.03250,0.04791,0.041202,0.053419,0.051189,0.041202,0.041825,0.036488


In [26]:
df['dt_s1'] = np.where(df['year'] == 2017, np.NaN, df['score_1'] - df['score_0'])
df['dt_s2'] = np.where(df['year'] >= 2016, np.NaN, df['score_2'] - df['score_1'])
df['dt_z1'] = np.where(df['year'] == 2017, np.NaN, df['z_score_1'] - df['z_score_0'])
df['dt_z2'] = np.where(df['year'] >= 2016, np.NaN, df['z_score_2'] - df['z_score_1'])
df['dt_s1a'] = np.where(df['year'] == 2017, np.NaN, (df['score_1'] - df['score_0'])
                        - (df['s_1_a'] - df['s_0_a']))
df['dt_s2a'] = np.where(df['year'] >= 2016, np.NaN, (df['score_2'] - df['score_1'])
                        - (df['s_2_a'] - df['s_1_a']))
df['dt_z1a'] = np.where(df['year'] == 2017, np.NaN, (df['z_score_1'] - df['z_score_0'])
                        - (df['z_1_a'] - df['z_0_a']))
df['dt_z2a'] = np.where(df['year'] >= 2016, np.NaN, (df['z_score_2'] - df['z_score_1'])
                        - (df['z_2_a'] - df['z_1_a']))

In [27]:
df.set_index(['gvkey1', 'year'], inplace=True)

In [28]:
dt_sim = df.rename(columns={'n_score_0':'n0',
                            'n_score_1':'n1',
                            'n_score_2':'n2'})[['fyear','permno','datadate',
                                                'n0','n1','n2',
                                                'dt_s1','dt_s2','dt_z1','dt_z2',
                                                'dt_s1a','dt_s2a','dt_z1a','dt_z2a']]

Number of observations with non-missig _dt avg_sim_ and adjusted _dt avg_sim_ in t1 & t2

In [45]:
dt_sim.to_csv('../2_pipeline/get_these.csv')

## Control

_IBES_
- Change in number of analysts

_CRSP_
- Ret. volatility
- MTB

_Compustat_
- Net asset growth
- Income
- Assets
- Age
- R&D
- Loss

_Ravenpack DJ, PR_
- Number of articles

Cross-section
- M&A result
- Differentiation

Run _1\_code/cko.sas_ on WRDS using the following file

In [46]:
!scp ../2_pipeline/get_these.csv hohn@wrds-cloud.wharton.upenn.edu:/scratch/ou/hohn/

get_these.csv                                 100%   19MB   1.7MB/s   00:11    


In [50]:
!scp hohn@wrds-cloud.wharton.upenn.edu:/scratch/ou/hohn/compcontrol.csv ../0_data/external/compcontrol.csv

compcontrol.csv                               100%   10MB   1.4MB/s   00:07    


In [51]:
dt_sim.head()

fyear permno   datadate  n0  n1  n2     dt_s1     dt_s2  \
gvkey1 year                                                             
1004   1995  1994.0  54594 1995-05-31   1   1   1  0.117300 -0.053500   
       1996  1995.0  54594 1996-05-31  10   8   7  0.022458  0.005370   
       1997  1996.0  54594 1997-05-31  10   9   7 -0.018204  0.010216   
       1998  1997.0  54594 1998-05-31  10   6   5  0.003790  0.016460   
       1999  1998.0  54594 1999-05-31  10  10  10  0.002240  0.015410   

               dt_z1    dt_z2    dt_s1a    dt_s2a    dt_z1a    dt_z2a  
gvkey1 year                                                            
1004   1995  0.11730 -0.05350       NaN       NaN       NaN       NaN  
       1996  0.01380 -0.00057  0.018063  0.009105  0.023626  0.008049  
       1997 -0.02096  0.00164 -0.029404  0.014351 -0.026117  0.021779  
       1998 -0.00665  0.00562 -0.005035  0.009124  0.006675  0.013681  
       1999  0.00224  0.01541 -0.009977  0.017640  0.001617  0.020747

In [60]:
controls=['permno', 'cusip','fyear','atg','mb','income','r_d','age','neg_ni']

In [64]:
compcont = pd.read_csv('../0_data/external/compcontrol.csv', 
                       index_col=['gvkey1', 'datadate'])[controls]

In [72]:
compcont[compcont.cusip.isnull()]

permno cusip  fyear       atg        mb     income  \
gvkey1 datadate                                                          
1073   1996-12-31  10496.0   NaN   1996       NaN       NaN        NaN   
1108   1997-06-30  51510.0   NaN   1997       NaN       NaN        NaN   
1164   2002-12-31  11042.0   NaN   2002 -0.711255       NaN  -3.236321   
1183   1999-01-31  11181.0   NaN   1998       NaN  0.258312  -0.125984   
1259   2009-07-31  11739.0   NaN   2009 -0.895301       NaN  -8.494434   
...                    ...   ...    ...       ...       ...        ...   
266319 2008-12-31  90594.0   NaN   2008 -0.965551       NaN -16.389214   
277918 2012-12-31  91412.0   NaN   2012 -0.672912       NaN  -2.768838   
       2013-12-31  91412.0   NaN   2013 -0.819967       NaN  -5.666182   
       2014-12-31  91412.0   NaN   2014 -0.318182       NaN  -1.720533   
282189 2016-06-30  92215.0   NaN   2016       NaN       NaN        NaN   

                        r_d   age  neg_ni  
gvkey1 datadate                            
1073   1996-12-31       NaN   NaN     NaN  
1108   1997-06-30       NaN   NaN     NaN  
1164   2002-12-31  0.000000  29.0     1.0  
1183   1999-01-31  0.000000  17.0     1.0  
1259   2009-07-31       NaN  26.0     1.0  
...                     ...   ...     ...  
266319 2008-12-31       NaN   8.0     1.0  
277918 2012-12-31  0.729828  19.0     1.0  
       2013-12-31  0.000000  20.0     1.0  
       2014-12-31       NaN  21.0     1.0  
282189 2016-06-30       NaN   NaN     NaN  

[2058 rows x 9 columns]

## Link SDC to COMPUSTAT

### SDC and Compustat Link File

The link file is from [Michael Ewens](https://github.com/michaelewens/SDC-to-Compustat-Mapping.git). Cite papers below.

 ```
@article{phillips2013r,
  title={R\&D and the Incentives from Merger and Acquisition Activity},
  author={Phillips, Gordon M and Zhdanov, Alexei},
  journal={The Review of Financial Studies},
  volume={26},
  number={1},
  pages={34--78},
  year={2013},
  publisher={Society for Financial Studies}
  }
 ```

```
@article{ewensPetersWang2018,
 title={Acquisition prices and the measurement of intangible capital},
 author={Ewens, Michael and Peters, Ryan and Wang, Sean},
 journal={Working Paper}
 year={2018}
 }
```

### SDC processing prohibitively slow. Work on the WRDS cloud using _sdc_link.sas_.

In [73]:
col = ["DealNumber","agvkey","tgvkey","AMANAMES","ACUSIP",
       "APUBLIC","ATTITUDE","FORM","STATUSCODE", "EBITLTM",
       "AMV","ENTVAL","BOOKVALUE",
       "EQVAL","MV","NETASS","NILTM","PCT_CASH","PCT_STK",
       "PCT_OTHER","PCT_UNKNOWN","PR","RANKVAL","SALESLTM",
       "TMANAMES","TNATIONCODE","TPUBLIC","MASTER_CUSIP","TTICKER"]

In [74]:
type = {}
for var in col:
    if var in ["DealNumber","agvkey","tgvkey"]:
        type[var] = 'Int64'
    if var in ["ACUSIP","APUBLIC","ATTITUDE","FORM","STATUSCODE",
               "TNATIONCODE","TPUBLIC","MASTER_CUSIP","TTICKER"]:
        type[var] = 'category'


In [75]:
sdc_link['DATEANN'].max()

Timestamp('2017-09-13 00:00:00')

In [76]:
sdc_link = pd.read_csv('../0_data/external/sdc_gvkey.csv', 
                       header=0,
                       parse_dates=['DATEANN','DATEEFF','DATEFIN'],
                       dtype=type, low_memory=False)

In [77]:
for var in ["EBITLTM", "AMV","ENTVAL","BOOKVALUE",
            "EQVAL","MV","NETASS","NILTM","PR","RANKVAL","SALESLTM"]:
    sdc_link[var] = np.where(sdc_link[var].isin(['nan', 'None', 'P', 'M']), np.NaN,
                             sdc_link[var].str.replace(',',''))
    sdc_link[var] = pd.to_numeric(sdc_link[var]).astype('float')

In [78]:
sdc_link.set_index('DealNumber', inplace=True)

In [79]:
compmna = sdc_link[lambda x : ~x['FORM'].isin(['Acq. Part. Int.','Acq. Rem. Int.'])]

In [80]:
compmna[['FORM','agvkey']].groupby('FORM').count()

,agvkey
FORM,
Acq. Cert. Asts.,337
Acq. Maj. Int.,3244
Acq. Part. Int.,0
Acq. Rem. Int.,0
Acq. of Assets,54803
Acquisition,3
Merger,15771


Drop _AP_ and _AR_.

Form of the Transaction: 10 codes describing the specific form of the transaction:
- M (MERGER): A combination of business takes place or 100% of the stock of a public or private company is acquired.
- A (ACQUISITION): deal in which 100% of a company is spun off or split off is classified as an acquisition by shareholders.
- AM (ACQ OF MAJORITY INTEREST): the acquiror must have held less than 50% and be seeking to acquire 50% or more, but less than 100% of the target company’s stock.
- AP (ACQ OF PARTIAL INTEREST): deals in which the acquiror holds less than 50% and is seeking to acquire less than 50%, or the acquiror holds over 50% and is seeking less than 100% of the target company’s stock. 
- AR (ACQ OF REMAINING INTEREST): deals in which the acquiror holds over 50% and is seeking to acquire 100% of the target company’s stock.
- AA (ACQ OF ASSETS): deals in which the assets of a company, subsidiary, division, or branch are acquired. This code is used in all transactions when a company is being acquired and the consideration sought is not given.
- AC: (ACQ OF CERTAIN ASSETS): deals in which sources state that “certain assets” of a company, subsidiary, or division are acquired.
- R (RECAPITALIZATION): deals in which a company undergoes a shareholders’ Leveraged recapitalization in which the company issues a special one-time dividend (in the form of cash, debt securities, preferred stock, or assets) allowing shareholders to retain an equity interest in the company.
- B (BUYBACK): deals in which the company buys back its equity securities or securities convertible into equity, either on the open market, through privately negotiated transactions, or through a tender offer. Board authorized repurchases are included.
- EO (EXCHANGE OFFER): deals in which a company offers to exchange new securities for its equity securities outstanding or its securities convertible into equity.

In [81]:
datadate['lagdate'] = datadate.groupby('gvkey1')['datadate'].shift(1) + pd.DateOffset(days=1)

In [82]:
datadate['lagdate'] = np.where(datadate['lagdate'].isna(),
                              datadate['datadate'] - pd.DateOffset(years=1) + pd.DateOffset(days=1),
                              datadate['lagdate'])

In [83]:
compmna.to_sql('compmna', conn, index=True, if_exists='replace')
datadate.to_sql('datadate', conn, index = True, if_exists='replace')
qry = '''
    select *
    from 
        (select
                a.*, b.datadate, b.fyear
            from
                compmna a left join datadate b on
                a.agvkey == b.gvkey1 and b.datadate >= a.dateeff
            group by 
                a.DealNumber
        )
        '''
temp1 = pd.read_sql_query(qry, conn)

In [84]:
for col in ['tgvkey', 'fyear']:
    temp1[col] = temp1[col].astype('Int64')
for col in ['agvkey', 'tgvkey', 'ACUSIP', 'APUBLIC', 'ATTITUDE', 'FORM', 'STATUSCODE',
             'TNATIONCODE', 'TPUBLIC', 'MASTER_CUSIP', 'TTICKER']:
    temp1[col] = temp1[col].astype('category')
for col in ['datadate', 'DATEANN', 'DATEEFF', 'DATEFIN']:
    temp1[col] = temp1[col].astype('datetime64[ns]')

In [85]:
compmna = temp1[(temp1['datadate'] - temp1['DATEEFF']).dt.days <= 370]

col = list(compmna)
col.insert(5, col.pop(col.index('datadate')))
col.insert(6, col.pop(col.index('fyear')))
compmna = compmna[col]

In [86]:
compmna.set_index('DealNumber', inplace=True)

SDC obsevations with logical (less than 370 day difference from effective date) Compustat _datadate_

In [90]:
compmna

,agvkey,tgvkey,AMANAMES,ACUSIP,datadate,fyear,APUBLIC,ATTITUDE,FORM,STATUSCODE,...,PCT_OTHER,PCT_UNKNOWN,PR,RANKVAL,SALESLTM,TMANAMES,TNATIONCODE,TPUBLIC,MASTER_CUSIP,TTICKER
DealNumber,,,,,,,,,,,,,,,,,,,,,
194489020,1998,12347,"BANC ONE Corp,Columbus,Ohio",059438,1996-12-31,1996,Public,Friendly,Merger,C,...,NaN,NaN,20.15,695.476,417.411,"Premier Bancorp,Baton Rouge,LA",US,Public,740900,PRBC
336679020,9236,23979,Rhone-Poulenc Rorer Inc,76242T,1995-12-31,1995,Public,Friendly,Acq. Maj. Int.,C,...,NaN,100.0,25.00,150.000,1.540,Applied Immune Sciences Inc,US,Public,03820F,AISX
344050020,5048,NaN,General Electric Capital Corp,369622,1997-12-31,1997,Public,Friendly,Acq. of Assets,C,...,NaN,NaN,NaN,NaN,NaN,Trans-Leasing Intl-Assets,US,Sub.,89324H,NaN
368462040,8543,NaN,Philip Morris Cos Inc,718154,1998-12-31,1998,Public,Friendly,Acq. Maj. Int.,C,...,NaN,100.0,NaN,211.895,NaN,Tabaqueira Empresa Industrial,PO,Govt.,87327A,NaN
378271020,25341,NaN,Kronos Inc,501052,1999-09-30,1999,Public,Friendly,Acq. of Assets,C,...,NaN,NaN,NaN,NaN,NaN,Interboro Systems Corp,US,Priv.,45843L,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113578020,22037,NaN,Sientra Inc,82621J,2017-12-31,2017,Public,Friendly,Merger,C,...,69.15,NaN,0.32,9.529,6.615,Miramar Labs Inc,US,Public,60503C,MRLB
3128101020,28742,14170,BorgWarner Inc,099724,2017-12-31,2017,Public,Friendly,Merger,C,...,NaN,NaN,22.00,150.487,60.363,Sevcon Inc,US,Public,81783K,SEV
3136151020,24720,4156,Wabash National Corp,929566,2017-12-31,2017,Public,Friendly,Merger,C,...,NaN,100.0,21.00,675.407,300.791,Supreme Industries Inc,US,Public,868607,STS


Non-missing target sales + Missing target sales but target _gvkey_ available

## Construct IV by acquirer's _gvkey_ and _datadate_

### Grab Compustat Sales

In [ ]:
sale = comp[comp['sale'].notnull() & comp['sale']!=0][['sale']]

In [ ]:
compmna.to_sql('compmna', conn, index=True, if_exists='replace')

sale.to_sql('sale', conn, index=True, if_exists='replace')
qry = '''
        select a.*, b.sale as a_sale, b.fyear as amatch
        from compmna a left join sale b
        on a.agvkey = b.gvkey1 and b.fyear =  
        (select fy.fyear 
            from sale as fy 
            where a.agvkey = fy.gvkey1 and fy.fyear < a.fyear
            order by fy.fyear desc
            limit 1
        )
        '''
temp = pd.read_sql_query(qry, conn)
temp.to_sql('temp', conn, index=True, if_exists='replace')

qry = '''
        select a.*, b.sale as t_sale, b.fyear as tmatch
        from temp a left join sale b
        on a.tgvkey = b.gvkey1 and b.fyear =  
        (select fy.fyear 
            from sale as fy 
            where a.tgvkey = fy.gvkey1 and fy.fyear < a.fyear
            order by fy.fyear desc
            limit 1
        )
        '''
compmna = pd.read_sql_query(qry, conn)

In [ ]:
for col in ['agvkey', 'tgvkey']:
    compmna[col] = pd.to_numeric(compmna[col]).astype('Int64')
for col in ['fyear', 'amatch', 'tmatch']:
    compmna[col] = compmna[col].astype('Int64')
for col in ['agvkey', 'tgvkey', 'ACUSIP', 'APUBLIC', 'ATTITUDE', 'FORM', 'STATUSCODE',
             'TNATIONCODE', 'TPUBLIC', 'MASTER_CUSIP', 'TTICKER']:
    compmna[col] = compmna[col].astype('category')
for col in ['datadate', 'DATEANN', 'DATEEFF', 'DATEFIN']:
    compmna[col] = compmna[col].astype('datetime64[ns]')

In [ ]:
compmna.drop(columns=['index'], inplace=True)

In [ ]:
compmna.set_index(['agvkey','DealNumber'], inplace=True, verify_integrity=True)

### Calculate IV based on effective date

#### Number of days from effective date to fiscal year-end

In [ ]:
compmna['days'] = (compmna['datadate'] - compmna['DATEEFF']).dt.days

Very few observations have negative target and acquirer sales.

In [ ]:
compmna = compmna[~(compmna['a_sale']<0) & ~(compmna['t_sale']<0) & compmna['a_sale'].notnull()]

In [ ]:
compmna['iv1'] = compmna['days']/365 * (abs(compmna['SALESLTM']/compmna['a_sale']))

In [ ]:
compmna['iv4'] = abs(compmna['SALESLTM']/compmna['a_sale'])

In [ ]:
compmna['iv2'] = np.where(compmna['iv1'].notnull(), compmna['iv1'], 
                         compmna['days']/365 * (abs(compmna['MV']/compmna['AMV'])))

In [ ]:
compmna['iv5'] = np.where(compmna['iv4'].notnull(), compmna['iv4'],(abs(compmna['MV']/compmna['AMV'])))

In [ ]:
compmna['iv3'] = np.where(compmna['iv2'].notnull(), compmna['iv2'], 
                         compmna['days']/365 * (abs(compmna['t_sale']/compmna['a_sale'])))

In [ ]:
compmna['iv6'] = np.where(compmna['iv5'].notnull(), compmna['iv5'], (abs(compmna['t_sale']/compmna['a_sale'])))

In [ ]:
iv = compmna.reset_index().groupby(['agvkey','datadate']).sum()[['iv1','iv2','iv3','iv4','iv5','iv6']]

In [ ]:
iv['n'] = compmna.reset_index()[['agvkey','datadate','DealNumber']].groupby(['agvkey','datadate']).count()

In [ ]:
iv = iv[iv['n'].notnull()]

#### Change IV to NaN based on values

- Change 4 to NaN if 4 is 0 and 5 is not 
- Change 4, 5 to NaN if 4,5 are 0 and 6 is not
- Change 6 to NaN if 4-6 are 0

In [ ]:
iv['iv4'] = np.where((iv['iv4']==0)&(iv['iv5']>0),np.NaN, iv['iv4'])

In [ ]:
iv['iv4'] = np.where((iv['iv4']==0)&(iv['iv5']==0)&(iv['iv6']>0),
                     np.NaN, iv['iv4'])
iv['iv5'] = np.where((iv['iv4'].isna())&(iv['iv5']==0)&(iv['iv6']>0),
                     np.NaN, iv['iv5'])

In [ ]:
iv['iv4'] = np.where((iv['iv4']==0)&(iv['iv5']==0)&(iv['iv6']==0),
                     np.NaN, iv['iv4'])
iv['iv5'] = np.where((iv['iv4'].isna())&(iv['iv5']==0)&(iv['iv6']==0),
                     np.NaN, iv['iv5'])
iv['iv6'] = np.where((iv['iv4'].isna())&(iv['iv5'].isna())&(iv['iv6']==0),
                     np.NaN, iv['iv6'])

In [ ]:
iv['iv1'] = np.where(iv['iv4'].isna(), np.NaN, iv['iv1'])
iv['iv2'] = np.where(iv['iv5'].isna(), np.NaN, iv['iv2'])
iv['iv3'] = np.where(iv['iv6'].isna(), np.NaN, iv['iv3'])

Number of each IV

In [ ]:
print(len(iv[iv['iv1'].notnull()]),len(iv[iv['iv2'].notnull()]),len(iv[iv['iv3'].notnull()]))

## Combine _dt_sim_ and _iv_

In [ ]:
sim_iv = dt_sim[dt_sim['dt_s1'].notnull()].reset_index().rename(columns={'gvkey1':'agvkey'}).merge(iv.reset_index(),
                                                                        on=['agvkey','datadate'],
                                                                        indicator=True, how='left').set_index(['agvkey','datadate'])

In [ ]:
sim_iv.head()

In [ ]:
len(sim_iv[lambda x : (x['fyear']>=1997)&(x['fyear']<=2012)&(x['n0']==10)&(x['_merge']=='left_only')])+len(sim_iv[lambda x : (x['fyear']>=1997)&(x['fyear']<=2012)&(x['n0']==10)&(x['_merge']=='both')])

In [ ]:
len(sim_iv[lambda x : (x['fyear']>=1997)&(x['fyear']<=2012)&(x['n0']==10)&(x['_merge']=='left_only')])

In [ ]:
len(sim_iv[lambda x : (x['fyear']>=1997)&(x['fyear']<=2012)&(x['n0']==10)&(x['_merge']=='both')])

In [ ]:
len(sim_iv[lambda x : (x['fyear']>=1997)&
           (x['fyear']<=2012)&
           (x['n0']==10)&
           (x['_merge']=='both')&
           (x['iv2'].notnull())])

In [ ]:
np.mean(sim_iv[(sim_iv['_merge']=='left_only')&(sim_iv['n0']==10)])

In [ ]:
np.mean(sim_iv[(sim_iv['_merge']=='both')&(sim_iv['n0']==10)])

### M&A Disclosure

In [ ]:
disc = pd.read_csv('~/Dropbox/Project/cko/0_data/manual/disc.csv', parse_dates=['DATADATE'])
disc['CIK'] = disc['CIK'].apply(lambda x: str(int(x)).zfill(10) if pd.notnull(x) else None)

In [ ]:
temp = sim_iv[lambda x : (x['fyear']>=1997)&
           (x['fyear']<=2012)&
           (x['n0']==10)&
           (x['_merge']=='both')&
           (x['iv2'].notnull())]

In [ ]:
disc.head()

In [ ]:
disc = disc.rename(columns={'GVKEY':'agvkey','DATADATE':'datadate'}).set_index(['agvkey','datadate'])

In [ ]:
temp.merge?

In [ ]:
temp.drop(columns=['_merge']).merge(disc,left_index=True,right_index=True,how='outer',indicator=True)[lambda x : x['_merge']=='both']

In [ ]:
iv = iv.join(disc.rename(columns={'GVKEY':'agvkey', 'DATADATE':'datadate'}).set_index(['agvkey', 'datadate']),\
        on=['agvkey', 'datadate'], how='left')

In [ ]:
iv[(iv['MD_A']==5)&(iv['disc']==0)]

In [ ]:
iv['disc'] = np.where((iv['MD_A']>=3)&(iv['PROFORMA'].isin(['I','C'])), 1, 0)

Disclosure also might need additonal data collection.

## Private Target Data (Chen 2019)

The _encoding_ option allows proper string imports.

In [ ]:
cw = pd.read_sas('../0_data/manual/CW2019.sas7bdat', format = 'sas7bdat', encoding="utf-8")

In [ ]:
cw

In [ ]:
cw[['gvkey','ATTITUDE']].groupby('ATTITUDE').count()

In [ ]:
cw['gvkey1'] = pd.to_numeric(cw['gvkey']).astype('int64')

_priv_ dataset is a subset of _compmna_ that will be matched to Ciao-Wei's data.

In [ ]:
priv = sdc[(sdc['RANKVAL'].notnull()) & (sdc['TPUBLIC'] == 'Priv.')]
priv.drop_duplicates(inplace=True)
priv = priv[(priv['DATEEFF'] >= '1997-01-01') & (priv['DATEEFF'] <= '2013-12-31')]
private = priv.compute()

for var in ['DATEANN', 'DATEEFF']:
    private[var] = pd.to_datetime(private[var]).astype('datetime64[ns]')

The code below grabs all but 5 _MASTER_DEAL_NO_ from _private_.

In [ ]:
# merge by dates and names
cw = cw.merge(private[['AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES', 'MASTER_DEAL_NO']],
         left_on=['ACQ_NAME', 'ANN', 'EFF', 'TRG_NAME'],
         right_on=['AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES'], how='left')
col = list(cw)
col.insert(0, col.pop())
cw = cw[col]

cw.drop(columns=['AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES'], inplace=True)

cw.drop_duplicates(inplace=True)

# review dates and names of the missing
missing = private[['MASTER_DEAL_NO', 'AMANAMES', 'DATEANN', 'DATEEFF', 'TMANAMES']].merge(cw[cw['MASTER_DEAL_NO'].isna()][['ANN', 'EFF', 'ACQ_NAME', 'TRG_NAME']],
                                                           how='right',left_on=['DATEANN', 'DATEEFF'], right_on=['ANN', 'EFF'])

missing.drop_duplicates(inplace=True)
missing.reset_index(drop=True, inplace=True)

# manual match
missing = missing.iloc[[0, 1, 5, 16, 24, 26, 28, 29, 39, 44, 53]][['MASTER_DEAL_NO', 'ANN', 'EFF', 'ACQ_NAME', 'TRG_NAME']]

# update MASTER_DEAL_NO
cw = cw.merge(missing, 
         left_on=['ACQ_NAME', 'ANN', 'EFF', 'TRG_NAME'],
         right_on=['ACQ_NAME', 'ANN', 'EFF', 'TRG_NAME'], how='left', suffixes=('','_y'))

cw['MASTER_DEAL_NO'] = np.where(cw['MASTER_DEAL_NO'].isna(), cw['MASTER_DEAL_NO_y'], cw['MASTER_DEAL_NO'])

cw.drop(columns=['MASTER_DEAL_NO_y'], inplace=True)

In [ ]:
col = list(cw)
col.insert(1, col.pop())
cw = cw[col]

In [ ]:
cw.to_sql('compmna', conn, index=False, if_exists='replace')
datadate.to_sql('datadate', conn, index = True, if_exists='replace')
qry = '''
    select  
        a.*, b.datadate
    from
        compmna a join datadate b on
        a.gvkey1 == b.gvkey1 and a.EFF between b.lagdate and b.datadate 
    '''
cw = pd.read_sql_query(qry, conn)

In [ ]:
cw['datadate'] = pd.to_datetime(cw['datadate']).astype('datetime64[ns]')

cw['year'] = cw['datadate'].dt.year

In [ ]:
df.to_sql('avg_sim', conn, index=False, if_exists='replace')
cw.to_sql('cw', conn, index = False, if_exists='replace')
qry = '''
    select  
        a.*
    from
        avg_sim a join (select distinct gvkey1, year from cw) b
        on a.gvkey1 = b.gvkey1 and a.year = b.year
    '''
cw_sim = pd.read_sql_query(qry, conn)

In [ ]:
cw_sim = cw_sim[['gvkey1', 'year', 'dt_s1', 'dt_z1', 'dt_s2', 'dt_z2']]

In [ ]:
tnic_industry.to_sql('tnic', conn, index=True, if_exists='replace')
cw_sim.to_sql('cw_sim', conn, index = False, if_exists='replace')
qry = '''
    select  
        a.gvkey1, a.year, a.gvkey2
    from
        tnic a join (select distinct gvkey1, year from cw_sim) b
        on a.gvkey1 = b.gvkey1 and a.year = b.year
    '''
cw_tnic = pd.read_sql_query(qry, conn)

In [ ]:
cw_tnic = cw_tnic.merge(df[['gvkey1', 'year', 'dt_s1', 'dt_s2', 'dt_z1', 'dt_z2']]
                        , left_on=['gvkey2', 'year'], right_on=['gvkey1', 'year'])

In [ ]:
cw_tnic.drop(columns=['gvkey1_y'], inplace=True)
cw_tnic.rename(columns={'gvkey1_x':'gvkey1'}, inplace=True)

In [ ]:
tnic_avg = cw_tnic.groupby(['gvkey1', 'year']).mean().drop(columns=['gvkey2'])

In [ ]:
cw_sim = cw_sim.merge(tnic_avg, left_on=['gvkey1', 'year'], right_on=['gvkey1', 'year'])

In [ ]:
cw_sim['dt_s1'] = cw_sim['dt_s1_x'] - cw_sim['dt_s1_y']
cw_sim['dt_z1'] = cw_sim['dt_z1_x'] - cw_sim['dt_z1_y']
cw_sim['dt_s2'] = cw_sim['dt_s2_x'] - cw_sim['dt_s2_y']
cw_sim['dt_z2'] = cw_sim['dt_z2_x'] - cw_sim['dt_z2_y'] 
# cw_sim = cw_sim[['gvkey1', 'year', 'dt_s1', 'dt_z1', 'dt_s2', 'dt_z2']]

In [ ]:
cw = cw.merge(cw_sim, left_on=['gvkey1', 'year'], right_on=['gvkey1', 'year'])

In [ ]:
cw.to_stata('/Users/ohn0000/Dropbox/Project/cko/2_pipeline/cw.dta')

### Materiality of M&A

In [ ]:
material = pd.read_csv('/Users/ohn0000/Project/cko/0_data/external/materiality.csv')
material.set_index(["year", "gvkey1"], inplace=True, verify_integrity=True)

Be careful since the _year_ here refers to the M&A firm-year. The _year_ in __avg_sim__ is the year competitors are identified. 

In [ ]:
manual = disc.join(material)[['DATADATE', 'CIK', 'TGTAT_ACQAT', 'TGTDVAL_ACQAT', 'MD_A', 'PROFORMA']].sort_index()

In [ ]:
# import wrds
# db = wrds.Connection(wrds_username = "yaera")

# ma_details_desc = db.describe_table('sdc', 'ma_details').sort_values('name')
# with pd.option_context('display.max_rows', None):
#     print(ma_details_desc)

|     Variable | Description                    |
|:------------:|:-------------------------------|
|bookvalue     |Target Book Value (\$mil)       |
|compete       |Competing Bidder (Y/N)          |
|competecode   |Competing Bid Deal Code         |  
|dateann       |Date Announced                  |
|dateannest    |_dateann_ is estimated (Y/N)    | 
|dateeff       |Date Effective                  | 
|ebitltm       |Target EBIT LTM (\$mil)         |
|pct_cash      |Percentage of consideration paid in cash|
|pct_other|Percentage of consideration paid in other then cash or stock|
|pct_stk|Percentage of consideration paid in stock|
|pct_unknown|Percentage of consideration which is unknown|
|ptincltm|Target Pre-Tax Income LTM (\$mil)|
|salesltm|Target Sales LTM (\$mil)|
|rankval|Ranking Value incl Net Debt of Target (\$mil)|

Run sql query below on _WRDS_

In [ ]:
# import wrds
# sdc_query = """
# select master_deal_no as DealNumber, 
#         bookvalue, 
#         compete, 
#         competecode, 
#         dateann, 
#         dateannest, 
#         dateeff, 
#         ebitltm, 
#         pct_cash,
#         pct_other,
#         pct_stk,
#         pct_unknown,
#         ptincltm,
#         salesltm,
#         rankval
# from sdc.ma_details
# where dateeff is not null 
# """
# # and master_deal_no in %(deal_no)s
# sdc = db.raw_sql(sdc_query, date_cols=['dateann', 'dateeff'])
# sdc.to_pickle('/home/upenn/yaera/sdc.pkl')

In [ ]:
sdc = pd.read_pickle('/Users/ohn0000/Project/cko/0_data/external/sdc.pkl')
sdc.drop_duplicates('dealnumber', inplace = True)
sdc['dealnumber'] = sdc['dealnumber'].astype('int64')

# clear up values and change dtype to 'float'
for column in ['bookvalue', 'ebitltm', 'pct_cash', 'pct_other', 'pct_stk', 'pct_unknown', 'ptincltm', 'salesltm', 'rankval']:
    sdc[column] = sdc[column].apply(lambda x: np.NaN if x == '*********' else (np.NaN if pd.isna(x) else (float(x.replace(',', '')) if isinstance(x, str) else float(x))))
    sdc[column].astype('float16')

In [ ]:
sdc_sub = pd.merge(sdc_link, sdc,
                   left_index=True, right_on='dealnumber').drop('dealnumber', axis='columns')
sdc_sub.index.name = 'dealnumber'

In [ ]:
sdc_sub.sort_values(['agvkey', 'dateeff'], inplace=True)

In [ ]:
sdc_link['agvkey'].count() / sdc['dealnumber'].count()

In [ ]:
sdc_sub['agvkey'].count() / sdc_link['agvkey'].count() 

In [ ]:
sdc_sub.profile_report(style={'full_width':True})

Use __compustat__ _datadate_ and gvkey to link the sdc data to the similarity scores

In [ ]:
import wrds
db = wrds.Connection(wrds_username = 'hohn')

sdc_quary = """
select gvkey, datadate, fyear, cusip,  cik
from comp.funda
where consol = %(consol)s and indfmt in %(indfmt)s and datafmt = %(datafmt)s and popsrc = %(popsrc)s and curcd in %(curcd)s
"""

parm = {'consol':('C'), 'indfmt' : ('INDL', 'FS'), 'datafmt': ('STD'), 'popsrc' : ('D'), 'curcd' : ('USD', 'CAD')}

funda = db.raw_sql(sdc_quary, params = parm, date_cols = ['datadate'])

In [ ]:
funda['start'] = funda['datadate'] - pd.DateOffset(months = 12) + pd.DateOffset(days = 1)
funda['gvkey'] = funda['gvkey'].astype('int64')
funda.set_index('gvkey', inplace=True)

In [ ]:
funda.fyear = funda.fyear.astype('Int16')

In [ ]:
import pandasql as ps

sql_query = '''
select a.*, b.datadate, b.fyear, b.cusip, b.cik
from sdc_sub a left join funda b
on a.agvkey = b.gvkey and a.dateeff between b.start and b.datadate
'''

newdf = ps.sqldf(sql_query, locals())

In [ ]:
col = list(newdf)
for i in range(2, 6):
    col.insert(i, col.pop(-1))
newdf = newdf.loc[:,col]

In [ ]:
for i in ['datadate', 'dateann', 'dateeff']:
    newdf[i] = newdf[i].astype('datetime64[ns]')
    
newdf['year'] = newdf['datadate'].dt.year.astype('Int16')
for i in ['fyear', 'agvkey', 'tgvkey']:
    newdf[i] = newdf[i].astype('Int64')

In [ ]:
col = list(newdf)
col.insert(col.index('datadate'), col.pop(col.index('year')))
newdf = newdf.loc[:,col]

In [ ]:
newdf = newdf.drop_duplicates(subset='dealnumber')

In [ ]:
newdf[newdf['agvkey'].notnull()]

In [ ]:
newdf['rankval'].count()

18994 observations with non-missing _rankval_

In [ ]:
newdf['salesltm'].count()

8055 observations with non-missing _salesltm_

In [ ]:
np.sum(newdf['rankval'].notnull() & newdf['salesltm'].notnull())

6445 observations with both _rankval_ and _salesltm_ available

## Append similarity score between acquirer and target

In [ ]:
upload = newdf[newdf['agvkey'].notnull() & newdf['tgvkey'].notnull() & newdf['year'].notnull()][['agvkey', 'tgvkey', 'year']].rename(columns={'agvkey':'gvkey1', 'tgvkey':'gvkey2'})
upload.to_csv('/Users/ohn0000/Project/cko/2_pipeline/upload.csv', index=False)
!scp /Users/ohn0000/Project/cko/2_pipeline/upload.csv $WRDS:/scratch/ou/hohn

Run this on wrds server. The __TNIC_All__ files should be uploaded in scratch beforehand.

In [ ]:
"""
The server killed the previous code that joins after combines all files. The current code instead loop over the files.
"""
# !cd /scratch/ou/hohn/TNIC_AllPairsDistrib
# !cat tnicall1996.txt > tnicall_combined.txt
# !for file in tnicall{1997..2017}.txt; do sed '1d' $file >> tnicall_combined.txt; done
# !cd ~


"""
atsim.py
"""


In [ ]:
!scp atsim.py $WRDS:~

In [ ]:
!scp $WRDS:/scratch/ou/hohn/atsim.csv /Users/ohn0000/Project/cko/2_pipeline/

In [ ]:
col = list(newdf)
col.insert(col.index('bookvalue'), col.pop(col.index('atsim')))
newdf = newdf.loc[:,col]

## IV candidates

The materiality measure based on deal value will be the last resort for the IV.   
Alternatively, 2SLS using multiple IVs is feasible.

Candidates
* Max deal value
* Sum deal value
* Datedif between _dateeff_ and _datadate_
    * _dateeff_ of the first M&A
    * _dateeff_ of the largest M&A
    * weighted average of _dateeff_ 

## Cross-sections
* Similarity between acquirer and target 
    - Relation stronger in diversifying
    - Could be more of a U-shaped relation, i.e., competitors don't follow when you move far enough
* Average value of pre-similarities between acquirer and close competitors 
    - Prediction not clear
* M&A performance during the completed firm-year
    - Relation stronger when M&A was more successful <-> how do we define success of an M&A?
* Number of close competitors of the target
    - Potential targets are candidates of future mergers
* How many competitors were there initially?
    - The size of the TNIC industry